# Feature Engineering
Explanation:

## Import Packages

In [1]:
from importnb import Notebook
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process
from transformers import pipeline
from fuzzywuzzy import fuzz
from dateutil import parser
from datetime import datetime
import re

/Users/janlinzner/Library/Mobile Documents/com~apple~CloudDocs/Documents/McGill/1_Lectures/Decision Analytics (MGSC 662)/2_Coding/Exercises/venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Dataframe import from 'DataPreprocessing'
Importing the required data frames from 'DataPreProcessing_Pipeline.ipynb'.

In [2]:
with Notebook():
    from DataPreprocessing_Pipeline import companies, funding, investors, founders

## New Features: Companies <> Investment Rounds

### Feature 1: Months Until First Round
Number of months between the funding date and the first investment round. Explanation: The table 'all_rounds' has every investment round made during the existence of the companies. We already mapped the respectively funding round to the startup. 

In [3]:
def calculate_months_until_first_round(company_id, founded_date):
    rounds = funding[funding['Company ID'] == company_id]

    if rounds.empty:
        return np.nan

    first_round_date = rounds['Announced Date'].min()

    delta_months = (first_round_date.year - founded_date.year) * 12 + (first_round_date.month - founded_date.month)
    return max(delta_months, 0)

companies['Months until First Round'] = companies.apply(
    lambda row: calculate_months_until_first_round(row['ID'], row['Founded Date']),
    axis=1
)

companies['Months until First Round'] = companies['Months until First Round'].fillna(-1)

### Feature 2: Grant Y/N
If the company received a Grant, they get a Yes (1). If not, a No (0). For VCs it is desirable to have a company with a Grant because it leverages the invested money as a Grant do not dilute existing shareholders.

In [4]:
def check_grant_for_company(company_id):
    company_rounds = funding[funding['Company ID'] == company_id]
    has_grant = any(company_rounds['Funding Type'] == 'Grant')
    return 1 if has_grant else 0

companies['Grant Y/N'] = companies['ID'].apply(check_grant_for_company)

### Feature 3: Last Round Type
This feature identifies the type of the most recent funding round for each company (e.g., Seed, Series A, etc.) or assigns "No Funding" to companies without any funding history.

In [5]:
# Convert the 'Announced Date' column in the funding data frame to datetime format
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])

# Sort the funding data by 'Company ID' and 'Announced Date' in ascending order
# Then group by 'Company ID' and select the last (most recent) funding round for each company
last_round = (
    funding.sort_values(by=['Company ID', 'Announced Date'])
    .groupby('Company ID')
    .last()
    .reset_index()
)

last_round = last_round[['Company ID', 'Funding Type']]

# Merge the most recent funding round data into the companies DataFrame
# Match rows on 'ID' in companies and 'Company ID' in last_round
companies = companies.merge(
    last_round,
    left_on='ID',
    right_on='Company ID',
    how='left'
)

companies = companies.rename(columns={'Funding Type': 'Last Round Type'})

# Drop the redundant 'Company ID' column after the merge
companies.drop(columns=['Company ID'], inplace=True)

# Fill missing values in 'Last Round Type' with 'No Funding' for companies with no funding data
companies['Last Round Type'] = companies['Last Round Type'].fillna('No Funding')

# Drop the existing 'Last Funding Type' column (if it exists) as it is replaced by the new feature
companies.drop(columns=['Last Funding Type'], inplace=True)


### Feature 4: Acquisition Status 'Was Acquired', 'Made Acquisitions'
- Was Acquired: A binary column (1 or 0) indicating whether the company was acquired.
- Made Acquisitions: A binary column (1 or 0) indicating whether the company made acquisitions.

In [6]:
# Check if the 'Acquisition Status' column exists in the companies data frame
if 'Acquisition Status' in companies.columns:
    unique_values = companies['Acquisition Status'].dropna().unique()
    print(unique_values)

['Was Acquired' 'Made Acquisitions' 'Made Acquisitions, Was Acquired']


In [7]:
# Initialize new binary columns to indicate acquisition activity
companies['Was Acquired'] = 0
companies['Made Acquisitions'] = 0

# Set 'Was Acquired' to 1 for rows where 'Acquisition Status' is 'Was Acquired'
companies.loc[companies['Acquisition Status'] == 'Was Acquired', 'Was Acquired'] = 1

# Set 'Made Acquisitions' to 1 for rows where 'Acquisition Status' is 'Made Acquisitions'
companies.loc[companies['Acquisition Status'] == 'Made Acquisitions', 'Made Acquisitions'] = 1

# For companies with 'Acquisition Status' as 'Made Acquisitions, Was Acquired',
# set both 'Made Acquisitions' and 'Was Acquired' to 1
companies.loc[
    companies['Acquisition Status'] == 'Made Acquisitions, Was Acquired',
    ['Made Acquisitions', 'Was Acquired']
] = 1

# Drop the original 'Acquisition Status' column as it is no longer needed
companies.drop(columns=['Acquisition Status'], inplace=True)

### Feature 5: Funding phases
- Project Funding: Indicates whether the company received early-stage funding like Angel, Pre-Seed, or Convertible Note.
- Startup Funding: Indicates whether the company received startup funding like Seed or Series Unknown.
- Growth Funding: Indicates whether the company received funding for growth, such as Series A, B, or C.
- Expansion Funding: Indicates whether the company received funding for expansion, such as Series D, E, or F.
- Exit Funding: Indicates whether the company received exit-related funding, such as Private Equity or Post-IPO rounds.

In [8]:
# Check if the 'Last Round Type' column exists in the companies data frame
if 'Last Round Type' in companies.columns:
    # Print distinct non-null values in the 'Last Round Type' column for reference
    distinct_funding_types = companies['Last Round Type'].dropna().unique()
    print(distinct_funding_types)
else:
    # Inform the user if the 'Last Round Type' column is missing
    print("The column 'Last Round Type' does not exist in the DataFrame.")


['Seed' 'No Funding' 'Venture - Series Unknown' 'Series A' 'Series B'
 'Convertible Note' 'Angel' 'Post-IPO Debt' 'Private Equity' 'Pre-Seed'
 'Grant' 'Series C' 'Series F' 'Series D' 'Post-IPO Secondary' 'Series E'
 'Post-IPO Equity']


In [9]:
# Define new columns for different funding stages and initialize them to 0
funding_columns = ['Project Funding', 'Startup Funding', 'Growth Funding', 'Expansion Funding', 'Exit Funding']
for col in funding_columns:
    companies[col] = 0

# Define a mapping of funding types to their respective funding stages
funding_mapping = {
    'Project Funding': ['Angel', 'Pre-Seed', 'Convertible Note', 'Grant', 'Venture - Series Unknown'],
    'Startup Funding': ['Seed', 'Venture - Series Unknown'],
    'Growth Funding': ['Series A', 'Series B', 'Series C'],
    'Expansion Funding': ['Series D', 'Series E', 'Series F'],
    'Exit Funding': ['Private Equity', 'Post-IPO Secondary', 'Post-IPO Equity']
}

# Assign binary values to funding stage columns based on the 'Last Round Type'
for funding_type, types in funding_mapping.items():
    companies.loc[companies['Last Round Type'].isin(types), funding_type] = 1

# Propagate funding stages hierarchically:
companies.loc[companies['Exit Funding'] == 1, funding_columns[:-1]] = 1

companies.loc[companies['Expansion Funding'] == 1, ['Growth Funding', 'Startup Funding', 'Project Funding']] = 1

companies.loc[companies['Growth Funding'] == 1, ['Startup Funding', 'Project Funding']] = 1

companies.loc[companies['Startup Funding'] == 1, 'Project Funding'] = 1

### Feature 6: HighFunding (Series B or higher)
High Funding: A binary indicator where...

- 1 indicates the company participated in a high funding round (e.g., Series B or above, Private Equity, or Post-IPO rounds).
- 0 indicates the company did not participate in these rounds.

In [10]:
high_funding_rounds = ['Series B', 'Series C', 'Series D', 'Series E', 'Series F', 'Private Equity', 'Post-IPO Equity']
companies['High Funding'] = companies['Last Round Type'].isin(high_funding_rounds).astype(int)

### Feature 7: Average Time To Next Round
The average time (in months) between funding events for a company, including the time from founding to the first funding round.

Value:
- -1: No valid funding data is available.
- x: Average time in months between funding rounds.

In [11]:
# Convert the 'Announced Date' column in the funding data frame to datetime format
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])

# Convert the 'Founded Date' column in the companies data frame to datetime format
companies['Founded Date'] = pd.to_datetime(companies['Founded Date'])

# Define a function to calculate the average time (in months) to the next funding round for a company
def calculate_avg_time_to_next_round(company_id, company_founded_date, funding_df):
    # Filter funding data for the specific company
    company_funding = funding_df[funding_df['Company ID'] == company_id]

    # Case 1: Only one funding round exists
    if len(company_funding) == 1:
        # Calculate time difference between the funding round and the founding date
        time_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        avg_time_to_next_round = time_diff / 30  # Convert days to months

    # Case 2: Two or more funding rounds exist
    elif len(company_funding) >= 2:
        time_diffs = []

        # Calculate time difference between the first funding round and founding date
        first_round_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        time_diffs.append(first_round_diff)

        # Sort funding rounds by their announcement date
        company_funding = company_funding.sort_values('Announced Date')

        # Calculate time differences between consecutive funding rounds
        for i in range(1, len(company_funding)):
            time_diff = (company_funding['Announced Date'].iloc[i] - company_funding['Announced Date'].iloc[i-1]).days
            time_diffs.append(time_diff)

        # Calculate the average time to the next round (convert days to months)
        avg_time_to_next_round = sum(time_diffs) / len(time_diffs) / 30

    # Case 3: No funding rounds exist
    else:
        avg_time_to_next_round = None

    # Handle invalid or missing data
    if pd.isna(avg_time_to_next_round) or avg_time_to_next_round < -1:
        avg_time_to_next_round = -1

    return avg_time_to_next_round

companies['Average Time To Next Round'] = companies.apply(
    lambda row: calculate_avg_time_to_next_round(row['ID'], row['Founded Date'], funding), axis=1
)

### Feature 8: Average Funding Size
The average size of funding rounds for a company (calculated as the total money raised divided by the number of funding rounds).

Value:
- 0: For companies with no funding data.
- Positive values representing the average size of funding rounds.

In [12]:
# Group the funding data frame by 'Company ID' to calculate summary statistics
funding_summary = funding.groupby('Company ID').agg(
    TotalMoneyRaised=('Money Raised', 'sum'),  # Sum of all money raised by the company
    TotalRounds=('Money Raised', 'count')      # Count of funding rounds
).reset_index()

# Calculate the average funding size for each company
funding_summary['Average Funding Size'] = funding_summary['TotalMoneyRaised'] / funding_summary['TotalRounds']

# Merge the funding summary into the companies data frame
# Match rows on 'ID' in companies and 'Company ID' in funding_summary
companies = companies.merge(
    funding_summary[['Company ID', 'Average Funding Size']],
    left_on='ID',
    right_on='Company ID',
    how='left'
)

# Fill missing values in 'Average Funding Size' with 0 for companies with no funding data
companies['Average Funding Size'] = companies['Average Funding Size'].fillna(0)

# Drop the redundant 'Company ID' column after the merge
companies.drop(columns=['Company ID'], inplace=True)

## New Features: Companies <> Investors

### Feature 1-4: Average Number of Investments by Investors
- Average Number of Investments by Investors
- Average Number of Exits by Investors
- Average Number of Lead Investments by Investors
- Average Number of Portfolio Organizations by Investors

In [13]:
# Explode the 'Investor Names' column in the funding data frame
funding_exploded = funding.explode('Investor Names')

# Remove leading and trailing whitespace from the 'Investor Names' column
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()

# Remove leading and trailing whitespace from the 'Organization/Person Name' column in the investors DataFrame
investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

# Merge the funding data with the investors data
merged = funding_exploded.merge(
    investors,
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

# List of columns containing numeric data to process
columns_to_process = ['Number of Investments', 'Number of Exits', 'Number of Lead Investments', 'Number of Portfolio Organizations']

# Convert the numeric columns to proper numeric data types, coercing invalid values to NaN
for col in columns_to_process:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')

# Identify investors not found in the investors DataFrame
not_found_investors = merged[merged['Number of Investments'].isnull()]['Investor Names'].unique()

# Print a message indicating the number of unmatched investors
if len(not_found_investors) > 0:
    print(f"{len(not_found_investors)} investors could not be found.")
else:
    print("All investors were successfully found.")

# Create a DataFrame of unmatched investors for reference or debugging
not_found_investors_df = pd.DataFrame(not_found_investors, columns=['Investor Names'])

# Remove the 'Company ID' column from companies if it exists to avoid duplication during the merge
if 'Company ID' in companies.columns:
    companies = companies.drop(columns=['Company ID'])

# Calculate averages for each numeric column by 'Company ID' and merge with the companies DataFrame
for col in columns_to_process:
    col_average = merged.groupby('Company ID')[col].mean().reset_index()

    # Merge
    companies = companies.merge(
        col_average,
        left_on='ID',
        right_on='Company ID',
        how='left',
        suffixes=('', '_drop')
    )

    # Replace NaN values with 0 for companies with no data for this column
    companies[col] = companies[col].fillna(0)

    # Rename the column to indicate it contains average data
    companies = companies.rename(columns={col: f'Average {col} by Investors'})

# Drop any temporary columns created during the merge (e.g., those ending with '_drop')
companies = companies.drop(columns=[col for col in companies.columns if col.endswith('_drop')], errors='ignore')


3608 investors could not be found.


### Feature 5: Origin Country of Investors
Investor Country: Indicates whether the company has any investors from the specified country:
- 1: At least one investor from the country is associated with the company.
- 0: No investors from the country are associated with the company.

In [14]:
# Explode the 'Investor Names' column in the funding data frame
funding_exploded = funding.explode('Investor Names')

# Remove leading and trailing whitespace from the 'Investor Names' and 'Organization/Person Name' column
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()

investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

# Merge the funding data with the investors data to include the country information
merged = funding_exploded.merge(
    investors[['Organization/Person Name', 'Country']],
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

# Get the first 5 distinct countries from the investors dataset for encoding
distinct_countries = investors['Country'].dropna().unique()[:5]

# Perform one-hot encoding of the 'Country' column in the merged DataFrame
country_encoded = pd.get_dummies(merged['Country'], prefix='Investor Country', dtype=int)

# Aggregate the one-hot encoded columns by 'Company ID'
# Use max() to ensure that if any investor from a given country is associated with a company, it gets a 1
country_aggregated = country_encoded.groupby(merged['Company ID']).max()

# Merge the aggregated country encoding data into the companies DataFrame
companies = companies.merge(
    country_aggregated,
    left_on='ID',
    right_index=True,
    how='left'
)

# Fill missing values for the encoded country columns with 0 (indicating no association with that country)
for country in distinct_countries:
    column_name = f'Investor Country: {country}'
    if column_name in companies.columns:
        companies[column_name] = companies[column_name].fillna(0)

### Feature 6: Top Investor Participation
Definition: A binary feature indicating whether a company's funding rounds included participation from top investors.

Values:
- 1: At least one top investor participated in the company's funding rounds.
- 0: No top investors participated in the company's funding rounds.

In [15]:
# Identify top investors based on the 'Number of Exits' column
# Top investors are defined as those with a number of exits greater than the mean
top_investors = investors[investors['Number of Exits'] > investors['Number of Exits'].mean()]['Organization/Person Name']

# Define a function to check if a company has participation from any top investor
def has_top_investor(company_id, funding_df, top_investors):

    # Filter the funding data for the specific company
    company_funding = funding_df[funding_df['Company ID'] == company_id]

    # If the company has no funding data, return 0 (no top investor participation)
    if company_funding.empty:
        return 0

    # Explode the 'Investor Names' column to handle multiple investors per funding round
    investors_list = company_funding['Investor Names'].explode()

    # Check if any of the company's investors are in the top investors list
    return 1 if any(investor in top_investors.values for investor in investors_list) else 0

# Apply the function to each company in the companies data frame
companies['Top Investor Participation'] = companies['ID'].apply(
    lambda x: has_top_investor(x, funding, top_investors)
)

## New Features: Companies

### Feature 1: Category One Hot Encoding
For each unique industry group in the Industry Groups column, a new binary feature is created:

Industry_<Group>:
- 1: The company is associated with the industry group <Group>.
- 0: The company is not associated with the industry group <Group>.

In [16]:
# Explode the 'Industry Groups' column to create one row per industry group
exploded = companies.explode('Industry Groups')

# Clean the 'Industry Groups' column by stripping whitespace and converting text to lowercase
exploded['Industry Groups'] = exploded['Industry Groups'].str.strip().str.lower()

# Filter out rows where 'Industry Groups' is missing or empty
exploded = exploded[exploded['Industry Groups'].notna() & (exploded['Industry Groups'] != '')]

# Perform one-hot encoding on the 'Industry Groups' column
one_hot_encoded = pd.get_dummies(exploded['Industry Groups'], prefix='Industry', dtype=int)

# Aggregate the one-hot encoded columns by the original index of the 'companies' DataFrame
# Use max() to ensure that if a company is associated with a specific industry group, it gets a 1
one_hot_aggregated = one_hot_encoded.groupby(exploded.index).max()

# Concatenate the aggregated one-hot encoded columns back to the original companies DataFrame
companies = pd.concat([companies, one_hot_aggregated], axis=1)

## New Features: LinkedIn-Founder (Aggregation to Company Level)

### Feature 1-6: Average LinkedIn Followers and Connections

- average_linkedin_followers_founders: Average number of LinkedIn followers of a company's founders.
- average_linkedin_connections_founders: Average number of LinkedIn connections of a company's founders.
- min_linkedin_followers_founders: Minimum number of LinkedIn followers among a company's founders.
- max_linkedin_followers_founders: Maximum number of LinkedIn followers among a company's founders.
- min_linkedin_connections_founders: Minimum number of LinkedIn connections among a company's founders.
- max_linkedin_connections_founders: Maximum number of LinkedIn connections among a company's founders.

In [17]:
def add_founder_metrics_to_companies(founders, companies):
    # Group the founders data frame by 'Company ID' and calculate various aggregate metrics
    founder_metrics = founders.groupby('Company ID').agg(
        average_linkedin_followers_founders=('followers', 'mean'),

        average_linkedin_connections_founders=('connections', 'mean'),

        min_linkedin_followers_founders=('followers', 'min'),

        max_linkedin_followers_founders=('followers', 'max'),

        min_linkedin_connections_founders=('connections', 'min'),

        max_linkedin_connections_founders=('connections', 'max')
    ).reset_index()

    # Merge the aggregated founder metrics into the companies data frame
    companies = companies.merge(founder_metrics, left_on='ID', right_on='Company ID', how='left')

    # Remove redundant or duplicate 'Company ID' columns from the merged DataFrame
    for col in ['Company ID', 'Company ID_y']:
        if col in companies.columns:
            companies.drop(columns=[col], inplace=True)

    # Return the updated companies DataFrame
    return companies

# Apply the function to add founder metrics to the companies data frame
companies = add_founder_metrics_to_companies(founders, companies)

## Feature 7-9: Highest Education

- Highest Education_Doctor/PhD: The number of founders in a company with a Doctorate/PhD degree.
- Highest Education_Master: The number of founders in a company with a Master's degree.
- Highest Education_Bachelor: The number of founders in a company with a Bachelor's degree.

In [18]:
def categorize_highest_education(founders):
    # Define a mapping of degree keywords to education levels
    education_mapping = {
        "Doctor/PhD": [
            "phd", "doctor", "doctoral", "dr.", "d.phil", "doctorate", "dsc",
            "dr.phil", "doctor of science", "dr.eng", "ph.d", "ed.d", "sc.d",
            "eng.d", "dr.med", "med.d", "doctor of medicine", "doctor of philosophy",
            "d.ed", "jd", "juris doctor", "doctor juris", "law doctorate", "d.v.m",
            "doctor of veterinary", "md", "m.d.", "doctor of law", "doctor of arts",
            "doctor in", "doctoral studies", "d.lit", "d.m.a", "d.clin.psych",
            "doctor of clinical psychology", "d.jur", "d.theol", "d.b.a",
            "doctor of business administration", "d.eng.sc", "d.arch", "d.d.s",
            "doctor of dental surgery", "d.v.sc", "d.med.sc", "d.p.h",
            "d.sc.tech", "doctor of public health", "d.health.sci", "d.n.p",
            "doctor of nursing practice", "doctor of social work", "doctor of theology",
            "d.comm", "doctor of communication", "d.env.sc", "doctor of environmental science"
        ],
        "Master": [
            "master", "m.sc", "msc", "mba", "m.tech", "ma", "m.eng", "m.ed",
            "ms", "m.phil", "mfa", "m.econ", "mfin", "master of science",
            "master of arts", "master of business", "master's degree",
            "m.des", "m.com", "m.div", "m.theol", "mres", "m.arch", "m.acc",
            "master of engineering", "master of finance", "master of commerce",
            "master of education", "master of philosophy", "master of public health",
            "mph", "m.pp", "m.ir", "mib", "m.int.business", "m.litt", "mchem",
            "diplom", "diploma", "diplom-ing", "diploma in engineering",
            "dipl.-ing", "dipl.-wirtschaftsingenieur", "dipl.-kfm", "dipl.-phys",
            "dipl.-math", "diplomkaufmann", "diplomingenieur", "diplomat",
            "m.comm", "m.a.ed", "m.sc.ed", "m.p.a", "m.h.a", "m.i.s", "m.c.s",
            "master of computer science", "master of information systems",
            "m.sc.tech", "m.plan", "master of planning", "mcm", "master of communication",
            "mhl", "master of human resources", "m.intl.rel", "master of international relations",
            "m.i.t", "m.arch.sc", "master of architecture", "m.journ", "m.fish.sci",
            "master of fisheries science", "m.r.s", "master of rural studies",
            "m.theo", "m.e.e", "master of electrical engineering", "m.med.sc",
            "m.env.sc", "master of environmental science", "m.agri", "master of agriculture",
            "m.p.h.a", "master of public health administration", "m.sc.math"
        ],
        "Bachelor": [
            "bachelor", "b.sc", "bsc", "b.tech", "ba", "b.eng", "b.ed",
            "bs", "b.arch", "bcom", "bba", "bfa", "bpharm", "b.econ",
            "bachelor's degree", "undergraduate", "bcom", "bdes", "bca",
            "bacc", "bachelor of science", "bachelor of arts", "bachelor of technology",
            "bachelor of engineering", "bachelor of education", "bachelor of commerce",
            "bachelor of fine arts", "bachelor of pharmacy", "bachelor of law",
            "bachelor of economics", "llb", "bachelor of computer applications",
            "b.litt", "b.a.e", "b.sc.eng", "b.plan", "b.comm", "b.h.sc",
            "b.i.t", "b.math", "b.stat", "b.mus", "b.of.design", "bcs",
            "bachelor of computing science", "b.eng.tech", "b.a.sc", "b.app.sci",
            "b.e", "b.journ", "b.a.hons", "b.sc.hons", "b.nurs", "b.sc.n",
            "b.soc.sc", "b.soc.work", "bachelor of social work", "b.v.sc",
            "bachelor of veterinary science", "b.med.sc", "b.biochem",
            "bachelor of biochemistry", "b.a.s", "b.env.sc", "bachelor of environmental science",
            "b.med", "bachelor of medicine", "b.optom", "bachelor of optometry",
            "b.psych", "bachelor of psychology", "b.public.health", "b.p.t",
            "bachelor of physical therapy"
        ]
    }

    # Define a priority mapping for education levels
    education_priority = {"Doctor/PhD": 3, "Master": 2, "Bachelor": 1}

    def get_highest_education(degree):
        degree_lower = str(degree).lower()
        for level, keywords in education_mapping.items():
            if any(keyword in degree_lower for keyword in keywords):
                return level
        return None

    # Categorize the level of each degree for degree_1 and degree_2
    founders['degree_1_level'] = founders['degree_1'].apply(get_highest_education)
    founders['degree_2_level'] = founders['degree_2'].apply(get_highest_education)

    # Map education levels to priorities for comparison
    founders['degree_1_priority'] = founders['degree_1_level'].map(education_priority).fillna(0)
    founders['degree_2_priority'] = founders['degree_2_level'].map(education_priority).fillna(0)

    # Determine the highest education level between degree_1 and degree_2
    founders['Highest Education'] = founders.apply(
        lambda row: row['degree_1_level'] if row['degree_1_priority'] >= row['degree_2_priority'] else row['degree_2_level'],
        axis=1
    )

    return founders

def add_highest_education_one_hot_to_companies(founders, companies):
    # Categorize the highest education level for each founder
    founders = categorize_highest_education(founders)

    # Perform one-hot encoding on the 'Highest Education' column
    one_hot_encoded = pd.get_dummies(founders['Highest Education'], prefix='Highest Education')

    # Add the one-hot encoded columns back to the founders data frame
    founders = pd.concat([founders, one_hot_encoded], axis=1)

    # Aggregate the one-hot encoded data by Company ID
    aggregated_one_hot = founders.groupby('Company ID')[one_hot_encoded.columns].sum().reset_index()

    # Merge the aggregated one-hot data into the companies DataFrame
    companies = companies.merge(aggregated_one_hot, left_on='ID', right_on='Company ID', how='left')

    # Fill missing values with 0 for companies with no associated founders
    companies.fillna(0, inplace=True)

    # Drop the redundant 'Company ID' column from the merged DataFrame
    companies.drop(columns=['Company ID'], inplace=True, errors='ignore')
    return companies

# Apply the function
companies = add_highest_education_one_hot_to_companies(founders, companies)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_3935/3398100639.py:103: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  companies.fillna(0, inplace=True)


In [19]:
def calculate_recognition_rate(founders):
    # Filter founders who have at least one degree
    total_with_degrees = founders[
        founders['degree_1'].notna() | founders['degree_2'].notna()
        ]

    # Count the number of degrees that are successfully recognized/mapped to education levels
    recognized_degrees = total_with_degrees['Highest Education'].notna().sum()

    # Count the total number of entries with at least one degree
    total_with_degrees_count = len(total_with_degrees)

    # Calculate the recognition rate as the proportion of recognized degrees
    recognition_rate = recognized_degrees / total_with_degrees_count if total_with_degrees_count > 0 else 0

    return recognition_rate, total_with_degrees_count, recognized_degrees

founders = categorize_highest_education(founders)

# Calculate the recognition rate and related statistics
recognition_rate, total_with_degrees, recognized_count = calculate_recognition_rate(founders)

print(f"Recognition Rate: {recognition_rate:.2%}")
print(f"Total Entries with Degrees: {total_with_degrees}")
print(f"Recognized Degrees: {recognized_count}")


Recognition Rate: 91.04%
Total Entries with Degrees: 1663
Recognized Degrees: 1514


### Feature 10: International Team
A binary feature indicating whether the company has at least one founder from a country other than Germany.

Values:
- 1: The company has at least one international founder.
- 0: All founders are from Germany, or no founder data is available for the company.

In [20]:
def add_international_team_to_companies(founders, companies):
    # Create a binary column in founders to indicate if a founder is international
    founders['is_international'] = founders['country_code'].apply(
        lambda x: 0 if x == 'DEU' or x is None else 1
    )

    # Group by 'Company ID' to determine if any founder in the company is international
    # Use max() to check if at least one founder is international
    international_team = founders.groupby('Company ID')['is_international'].max().reset_index()

    # Rename the column to 'International Team' for clarity
    international_team.rename(columns={'is_international': 'International Team'}, inplace=True)

    # Merge the 'International Team' data into the companies DataFrame
    companies = companies.merge(international_team, left_on='ID', right_on='Company ID', how='left')

    # Fill missing values with 0 for companies with no founders
    companies['International Team'].fillna(0, inplace=True)

    # Ensure the 'International Team' column is an integer type
    companies['International Team'] = companies['International Team'].astype(int)

    # Drop the redundant 'Company ID' column from the merged data frame if it exists
    if 'Company ID' in companies.columns:
        companies.drop(columns=['Company ID'], inplace=True)

    return companies

# Apply the function to add the 'International Team' feature to the companies data frame
companies = add_international_team_to_companies(founders, companies)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_3935/2678057080.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  companies['International Team'].fillna(0, inplace=True)


### Feature 11: Top University
The proportion of a company's founders who have a degree from a top university.

In [22]:
# New import because rapidfuzz should be used
from rapidfuzz import process, fuzz

# Load university ranking data
ranking = pd.read_csv('../../Datasets/Universities/UniversityRanking.csv')

# Convert the 'rank display' column to numeric, coercing errors to NaN
ranking['rank display'] = pd.to_numeric(ranking['rank display'], errors='coerce')

# Filter for the top 100 universities based on ranking
top100 = ranking[ranking['rank display'] <= 100]

# Extract a list of the top 100 university names
top_universities = top100['institution'].dropna().unique().tolist()

def get_top_university_match(uni, top_universities_list, threshold=90):
    if pd.isna(uni) or not uni.strip():
        return None, 0
    # Use RapidFuzz to find the best match in the list of top universities
    match = process.extractOne(uni, top_universities_list, scorer=fuzz.token_set_ratio)
    if match and match[1] >= threshold:
        return match[0], match[1]
    return None, 0

# Initialize columns for top university matching and scores
founders['degree_1_top_match'] = None
founders['degree_1_top_score'] = 0
founders['degree_2_top_match'] = None
founders['degree_2_top_score'] = 0

# Iterate through each founder and find matches for their degree universities
for idx, row in founders.iterrows():
    uni1 = row.get('degree_1_university', '')
    uni2 = row.get('degree_2_university', '')

    # Find the best match for each degree
    match1, score1 = get_top_university_match(uni1, top_universities)
    match2, score2 = get_top_university_match(uni2, top_universities)

    # Store the matches and scores in the DataFrame
    founders.at[idx, 'degree_1_top_match'] = match1
    founders.at[idx, 'degree_1_top_score'] = score1
    founders.at[idx, 'degree_2_top_match'] = match2
    founders.at[idx, 'degree_2_top_score'] = score2

print(founders[['degree_1_university', 'degree_1_top_match', 'degree_1_top_score',
                'degree_2_university', 'degree_2_top_match', 'degree_2_top_score']].head())

def check_top_university(universities, top_universities_list, threshold=90):
    for uni in universities:
        if pd.isna(uni) or not uni.strip():
            continue
        # Use RapidFuzz to find the best match in the list of top universities
        match = process.extractOne(uni, top_universities_list, scorer=fuzz.token_set_ratio)
        if match and match[1] >= threshold:
            return True
    return False

# Add a binary column to indicate if a founder has a degree from a top university
founders['founder_top_uni'] = founders.apply(
    lambda row: check_top_university(
        [row.get('degree_1_university', ''), row.get('degree_2_university', '')],
        top_universities
    ),
    axis=1
).astype(int)

# Group by 'Company ID' to calculate the proportion of founders from top universities
company_top_uni = founders.groupby('Company ID')['founder_top_uni'].mean().reset_index()

# Merge the top university data into the companies data frame
companies = companies.merge(company_top_uni, left_on='ID', right_on='Company ID', how='left')

# Fill missing values with 0 and rename the column for clarity
companies['Top University'] = companies['founder_top_uni'].fillna(0)

# Drop redundant columns after merging
companies.drop(['Company ID', 'founder_top_uni'], axis=1, inplace=True)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_3935/3171906205.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '93.87755102040816' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  founders.at[idx, 'degree_1_top_score'] = score1
/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_3935/3171906205.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '95.65217391304348' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  founders.at[idx, 'degree_2_top_score'] = score2


              degree_1_university degree_1_top_match  degree_1_top_score  \
0            ESCP Business School               None                 0.0   
2                             NaN               None                 0.0   
3  Technische Universität München               None                 0.0   
4                             NaN               None                 0.0   
5                             NaN               None                 0.0   

                        degree_2_university               degree_2_top_match  \
0  EBS Universität für Wirtschaft und Recht                             None   
2                                       NaN                             None   
3           Georgia Institute of Technology  Georgia Institute of Technology   
4                                       NaN                             None   
5                                       NaN                             None   

   degree_2_top_score  
0                 0.0  
2             

### Feature 12: Studies Abroad
The proportion of a company's founders who have a degree from a foreign university (outside Germany, Switzerland, and Austria).

Values:
- 1: All founders studied abroad.
- Between 0 and 1: Some but not all founders studied abroad.
- 0: No founders studied abroad, or no founder data is available.

In [23]:
# Filter universities outside Germany (DE), Switzerland (CH), and Austria (AT)
foreign_universities = ranking[~ranking['location code'].isin(['DE', 'CH', 'AT'])]

# Extract the list of foreign university names
foreign_universities = foreign_universities['institution'].dropna().unique().tolist()

def check_foreign_university(universities, foreign_universities_list, threshold=90):
    for uni in universities:
        if pd.isna(uni) or not uni.strip():
            continue
        # Use RapidFuzz to find the best match in the list of foreign universities
        match = process.extractOne(uni, foreign_universities_list, scorer=fuzz.token_set_ratio)
        if match and match[1] >= threshold:
            return True
    return False

# Add a binary column indicating whether a founder studied abroad
founders['founder_studies_abroad'] = founders.apply(
    lambda row: check_foreign_university(
        [row.get('degree_1_university', ''), row.get('degree_2_university', '')],  # Universities from degrees
        foreign_universities
    ),
    axis=1
).astype(int)

# Group by 'Company ID' to calculate the proportion of founders who studied abroad
company_abroad = founders.groupby('Company ID')['founder_studies_abroad'].mean().reset_index()

# Merge the proportion of founders who studied abroad into the companies data frame
companies = companies.merge(company_abroad, left_on='ID', right_on='Company ID', how='left')

# Fill missing values with 0 and rename the column for clarity
companies['Studies Abroad Founder'] = companies['founder_studies_abroad'].fillna(0)

# Drop redundant columns after merging
companies.drop(['Company ID', 'founder_studies_abroad'], axis=1, inplace=True)


## New Features: Companies <> LinkedIn Founder Information

In [24]:
# Load data set with additional information about founder
ln_filepath= '../Social Media Webscraping/Results/LinkedIn/Founders/founders_linkedin.csv'

ln_details = pd.read_csv(ln_filepath)

## Feature 1-3: First Time Founder
- firsttime_founder_ratio: The proportion of founders who are first-time founders.
- all_firsttime_founders: Indicator of whether all founders are first-time founders.
- any_firsttime_founder: Indicator of whether at least one founder is a first-time founder.

In [25]:
# Fill NaN values in ln_details with empty strings for consistent string processing
ln_details = ln_details.fillna("")

# Merge LinkedIn details (ln_details) with founders using the LinkedIn URL as the key
merged = ln_details.merge(founders, left_on="url", right_on="linkedin_url", how="left")

# Define a function to filter and extract job titles relevant to the founder's organization
def filter_job_titles(row):
    organization = row["Organization Name"]
    job_titles = []
    match_found = False

    # Loop through up to 10 job history records
    for i in range(1, 11):
        company_col = f"job_company_{i}"
        title_col = f"job_title_{i}"

        if company_col in row and title_col in row:
            company = row[company_col]
            if not match_found and fuzz.partial_ratio(organization.lower(), company.lower()) > 80:

                match_found = True
                continue
            if match_found and row[title_col]:
                job_titles.append(row[title_col])

    return " ".join(job_titles)

# Apply the filter_job_titles function to extract relevant job titles
merged["filtered_job_titles"] = merged.apply(filter_job_titles, axis=1)

# Load the zero-shot classification model for identifying founder roles
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def is_founder(row):
    job_titles = row["filtered_job_titles"]
    if not job_titles.strip():
        return 1

    # Classify job titles as 'founder' or 'not a founder'
    result = classifier(
        job_titles,
        candidate_labels=["founder", "not a founder"],
        hypothesis_template="This person is a {}."
    )

    # Return classification result
    return 0 if result["labels"][0] == "founder" else 1

# Apply the is_founder function to classify each person as a first-time founder
merged["is_firsttime_founder"] = merged.apply(is_founder, axis=1)

# Validate that the is_firsttime_founder column was successfully created
if "is_firsttime_founder" not in merged.columns:
    raise ValueError("is_firsttime_founder column was not created correctly in merged.")

# Display the first few rows of LinkedIn URL and first-time founder classification
print(merged[["linkedin_url", "is_firsttime_founder"]].head())

# Merge the first-time founder classification back into the founders data frame
founders = founders.merge(
    merged[["linkedin_url", "is_firsttime_founder"]],
    left_on="linkedin_url",
    right_on="linkedin_url",
    how="left"
)

# Validate that the is_firsttime_founder column was added to the founders data frame
if "is_firsttime_founder" not in founders.columns:
    raise ValueError("is_firsttime_founder column was not added to founders.")

# Calculate first-time founder statistics at the company level
company_firsttime_stats = founders.groupby('Company ID')['is_firsttime_founder'].agg(
    firsttime_founder_ratio='mean',
    all_firsttime_founders=lambda x: int(x.mean() == 1),
    any_firsttime_founder=lambda x: int(x.mean() > 0)
).reset_index()

# Ensure company and statistics data frame IDs are strings for merging
companies['ID'] = companies['ID'].astype(str)
company_firsttime_stats['Company ID'] = company_firsttime_stats['Company ID'].astype(str)

# Merge first-time founder statistics into the companies data frame
companies = companies.merge(company_firsttime_stats, left_on='ID', right_on='Company ID', how='left')

# Fill missing values for companies with no founder data
companies['firsttime_founder_ratio'] = companies['firsttime_founder_ratio'].fillna(0)
companies['all_firsttime_founders'] = companies['all_firsttime_founders'].fillna(0)
companies['any_firsttime_founder'] = companies['any_firsttime_founder'].fillna(0)

companies.drop(['Company ID'], axis=1, inplace=True)

# Rename columns for better readability
companies.rename(columns={
    'firsttime_founder_ratio': 'Firsttime Founder Ratio',
    'all_firsttime_founders': 'All Firsttime Founders',
    'any_firsttime_founder': 'Any Firsttime Founder'
}, inplace=True)

Device set to use mps:0


                                        linkedin_url  is_firsttime_founder
0  https://www.linkedin.com/in/johannes-stoffel-2...                     0
1              https://www.linkedin.com/in/zkaramat/                     0
2  https://www.linkedin.com/in/alexandre-gu%C3%A9...                     0
3                 https://www.linkedin.com/in/mituca                     1
4           https://www.linkedin.com/in/antoine-jeol                     0


## Feature 4-6: Researcher

- Researcher Ratio:Proportion of a company's founders classified as researchers.
- All Researchers: Binary indicator (1 or 0) denoting whether all founders in a company are researchers.
- Any Researcher: Binary indicator (1 or 0) denoting whether at least one founder in a company is a researcher.

In [27]:
# Fill missing values in LinkedIn details with empty strings for consistent processing
ln_details = ln_details.fillna("")

# Merge LinkedIn details (ln_details) with founders data using the LinkedIn URL as the key
merged = ln_details.merge(founders, left_on="url", right_on="linkedin_url", how="left")

def filter_job_titles(row):
    organization = row["Organization Name"]  # The founder's organization
    job_titles = []  # List to store relevant job titles
    match_found = False  # Flag to identify if the organization is matched

    # Iterate over up to 10 job history entries
    for i in range(1, 11):
        company_col = f"job_company_{i}"
        title_col = f"job_title_{i}"

        # Ensure the necessary columns exist
        if company_col in row and title_col in row:
            company = row[company_col]
            # Match the founder's organization with the job history company using string similarity
            if not match_found and fuzz.partial_ratio(organization.lower(), company.lower()) > 80:
                match_found = True
                continue
            # Collect job titles after the organization is matched
            if match_found and row[title_col]:
                job_titles.append(row[title_col])

    return " ".join(job_titles)

# Apply the filter_job_titles function to extract job titles for each row
merged["filtered_job_titles"] = merged.apply(filter_job_titles, axis=1)

# Load a zero-shot classification model for identifying researcher roles
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def is_researcher(row):
    job_titles = row["filtered_job_titles"]
    if not job_titles.strip():
        return 0

    result = classifier(
        job_titles,
        candidate_labels=["researcher", "not a researcher"],
        hypothesis_template="This person is a {}."
    )

    return 1 if result["labels"][0] == "researcher" else 0

# Apply the is_researcher function to classify each person as a researcher
merged["was_researcher"] = merged.apply(is_researcher, axis=1)

# Merge the researcher classification results back into the founders DataFrame
founders = founders.merge(
    merged[["linkedin_url", "was_researcher"]],
    left_on="linkedin_url",
    right_on="linkedin_url",
    how="left"
)

# Calculate researcher-related statistics at the company level
company_researcher_stats = founders.groupby('Company ID')['was_researcher'].agg(
    researcher_ratio='mean',
    all_researchers=lambda x: int(x.mean() == 1),
    any_researcher=lambda x: int(x.mean() > 0)
).reset_index()

# Ensure company IDs are strings for merging consistency
companies['ID'] = companies['ID'].astype(str)
company_researcher_stats['Company ID'] = company_researcher_stats['Company ID'].astype(str)

# Merge researcher statistics into the companies DataFrame
companies = companies.merge(company_researcher_stats, left_on='ID', right_on='Company ID', how='left')

# Fill missing values for companies with no founder data
companies['researcher_ratio'] = companies['researcher_ratio'].fillna(0)
companies['all_researchers'] = companies['all_researchers'].fillna(0)
companies['any_researcher'] = companies['any_researcher'].fillna(0)

# Drop redundant columns after merging
companies.drop(['Company ID'], axis=1, inplace=True)

# Rename columns for better readability
companies.rename(columns={
    'researcher_ratio': 'Researcher Ratio',
    'all_researchers': 'All Researchers',
    'any_researcher': 'Any Researcher'
}, inplace=True)


Device set to use mps:0


## Feature 7-9: Senior Roles
- Executive Ratio: Definition: Proportion of a company's founders who held executive roles before joining the company.
- All Executives: Definition: Binary indicator (1 or 0) denoting whether all founders in a company held executive roles.
- Any Executive: Binary indicator (1 or 0) denoting whether at least one founder in a company held an executive role.

In [28]:
from fuzzywuzzy import fuzz

# Fill missing values in LinkedIn details data frame with empty strings for consistency
ln_details = ln_details.fillna("")

# Merge LinkedIn details with founders data frame on the LinkedIn URL
merged = ln_details.merge(founders, left_on="url", right_on="linkedin_url", how="left")

def filter_pre_foundation_jobs(row):
    organization = row["Organization Name"]
    job_titles = []

    for i in range(1, 11):  # Loop through up to 10 job history records
        company_col = f"job_company_{i}"
        title_col = f"job_title_{i}"

        if company_col in row and title_col in row:
            company = row[company_col]
            # Stop collecting job titles once the founder's organization is found
            if fuzz.partial_ratio(organization.lower(), company.lower()) > 80:
                break
            # Add job title to the list if available
            if row[title_col]:
                job_titles.append(row[title_col])

    return " ".join(job_titles)

# Apply the filter_pre_foundation_jobs function to extract relevant job titles
merged["filtered_job_titles"] = merged.apply(filter_pre_foundation_jobs, axis=1)

# Load the zero-shot classification model for identifying executive roles
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def is_executive(row):
    job_titles = row["filtered_job_titles"]
    if not job_titles.strip():
        return 0

    # Classify job titles as 'executive' or 'not an executive'
    result = classifier(
        job_titles,
        candidate_labels=["executive", "not an executive"],
        hypothesis_template="This person is a {}."
    )

    return 1 if result["labels"][0] == "executive" else 0  # Return classification result

# Apply the is_executive function to classify each person based on their job titles
merged["was_executive"] = merged.apply(is_executive, axis=1)

# Validate that the was_executive column was successfully created
if "was_executive" not in merged.columns:
    raise ValueError("Column 'was_executive' was not created in merged.")

# Merge the executive classification results back into the founders data frame
founders = founders.merge(
    merged[["linkedin_url", "was_executive"]],
    left_on="linkedin_url",
    right_on="linkedin_url",
    how="left"
)

# Validate that the was_executive column was added to the founders data frame
if "was_executive" not in founders.columns:
    raise ValueError("Column 'was_executive' was not added to founders.")

# Calculate executive statistics at the company level
company_executive_stats = founders.groupby('Company ID')['was_executive'].agg(
    executive_ratio='mean',
    all_executives=lambda x: int(x.mean() == 1),
    any_executive=lambda x: int(x.mean() > 0)
).reset_index()

# Ensure company IDs are strings for merging consistency
companies['ID'] = companies['ID'].astype(str)
company_executive_stats['Company ID'] = company_executive_stats['Company ID'].astype(str)

# Merge executive statistics into the companies data frame
companies = companies.merge(company_executive_stats, left_on='ID', right_on='Company ID', how='left')

# Fill missing values for companies with no founder data
companies['executive_ratio'] = companies['executive_ratio'].fillna(0)
companies['all_executives'] = companies['all_executives'].fillna(0)
companies['any_executive'] = companies['any_executive'].fillna(0)

# Drop the redundant Company ID column after merging
companies.drop(['Company ID'], axis=1, inplace=True)

# Rename columns for better readability
companies.rename(columns={
    'executive_ratio': 'Executive Ratio',
    'all_executives': 'All Executives',
    'any_executive': 'Any Executive'
}, inplace=True)


Device set to use mps:0


## Feature 10-14: Job Durations
- Few Years Experience Ratio: Proportion of a company's founders with 1–5 years of total experience.
- Decade Experience Ratio: Proportion of a company's founders with 10+ years of total experience.
- Mid Career Experience Ratio: Proportion of a company's founders with 5–10 years of total experience.
- Avg Gaps in Experience: Average number of months of gaps in career experience per founder in a company.
- Avg Longest Position Duration: Average duration (in months) of the longest position held by a founder in a company.

In [29]:
# Define a dictionary to translate German month abbreviations to English
MONTHS_TRANSLATION = {
    "Jan.": "Jan", "Feb.": "Feb", "März": "Mar", "Apr.": "Apr",
    "Mai": "May", "Juni": "Jun", "Juli": "Jul", "Aug.": "Aug",
    "Sept.": "Sep", "Okt.": "Oct", "Nov.": "Nov", "Dez.": "Dec"
}

def translate_months(date_str):
    for de, en in MONTHS_TRANSLATION.items():
        date_str = date_str.replace(de, en)
    return date_str

def parse_intervals(row):
    intervals = []
    for col in row.index:
        if "duration" in col and isinstance(row[col], str) and row[col].strip():
            duration = translate_months(row[col])  # Translate German month names
            match = re.match(r"([\w.]+ \d{4}|\d{4})–(Heute|[\w.]+ \d{4}|\d{4})", duration)
            if match:
                start_str, end_str = match.groups()
                start_date = parser.parse(start_str, dayfirst=False)
                end_date = datetime.now() if end_str == "Heute" else parser.parse(end_str, dayfirst=False)
                intervals.append((start_date, end_date))
    return intervals

def merge_intervals(intervals):
    if not intervals:
        return []
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]
    for current in intervals[1:]:
        last = merged[-1]
        if current[0] <= last[1]:
            merged[-1] = (last[0], max(last[1], current[1]))
        else:
            merged.append(current)
    return merged

def calculate_total_months(merged_intervals):
    return sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_intervals)

def calculate_gaps(merged_intervals):
    if len(merged_intervals) < 2:
        return 0
    gaps = [
        (merged_intervals[i][0] - merged_intervals[i-1][1]).days // 30
        for i in range(1, len(merged_intervals))
    ]
    return sum(gaps)

# Extract job intervals, merge overlapping intervals, and calculate experience statistics
ln_details["intervals"] = ln_details.apply(lambda row: parse_intervals(row), axis=1)
ln_details["merged_intervals"] = ln_details["intervals"].apply(merge_intervals)
ln_details["total_experience_months"] = ln_details["merged_intervals"].apply(calculate_total_months)
ln_details["total_experience_years"] = ln_details["total_experience_months"] / 12
ln_details["few_years_experience"] = ln_details["total_experience_years"].apply(lambda x: 1 if 1 <= x <= 5 else 0)
ln_details["decade_experience"] = ln_details["total_experience_years"].apply(lambda x: 1 if x >= 10 else 0)
ln_details["mid_career_experience"] = ln_details["total_experience_years"].apply(lambda x: 1 if 5 <= x < 10 else 0)
ln_details["gaps_in_experience"] = ln_details["merged_intervals"].apply(calculate_gaps)
ln_details["longest_position_duration"] = ln_details["merged_intervals"].apply(
    lambda intervals: max((end - start).days / 30 for start, end in intervals) if intervals else 0
)

# Merge experience statistics into the founders data frame
founders = founders.merge(
    ln_details[["url", "few_years_experience", "decade_experience",
                "mid_career_experience", "gaps_in_experience",
                "longest_position_duration"]],
    left_on="linkedin_url",
    right_on="url",
    how="left",
    suffixes=("_founders", "_ln_details")
)

# Drop redundant columns
founders.drop(columns=["url"], inplace=True, errors="ignore")

# Aggregate experience statistics at the company level
agg_funcs = {
    "few_years_experience": "mean",
    "decade_experience": "mean",
    "mid_career_experience": "mean",
    "gaps_in_experience": "mean",
    "longest_position_duration": "mean"
}
company_experience_stats = founders.groupby("Company ID").agg(agg_funcs).reset_index()

# Rename aggregated columns for clarity
company_experience_stats.rename(
    columns={
        "few_years_experience": "few_years_experience_ratio",
        "decade_experience": "decade_experience_ratio",
        "mid_career_experience": "mid_career_experience_ratio",
        "gaps_in_experience": "avg_gaps_in_experience",
        "longest_position_duration": "avg_longest_position_duration"
    },
    inplace=True
)

# Merge company-level experience statistics into the companies data frame
companies["ID"] = companies["ID"].astype(str)
company_experience_stats["Company ID"] = company_experience_stats["Company ID"].astype(str)
companies = companies.merge(company_experience_stats, left_on="ID", right_on="Company ID", how="left")

# Fill missing values and drop redundant columns
companies.fillna(0, inplace=True)
companies.drop(columns=["Company ID"], inplace=True, errors="ignore")

# Rename final columns for readability
companies.rename(columns={
    'few_years_experience_ratio': 'Few Years Experience Ratio',
    'decade_experience_ratio': 'Decade Experience Ratio',
    'mid_career_experience_ratio': 'Mid Career Experience Ratio',
    'avg_gaps_in_experience': 'Avg Gaps in Experience',
    'longest_position_duration': 'Avg Longest Position Duration'
}, inplace=True)


### Final Removing of Columns

In [30]:
companies = companies.drop(['employeeCount', 'employeeCountRange', 'employeeCountRangeMin', 'employeeCountRangeMax', 'account_created', 'Company ID_x'], axis=1)

In [31]:
companies

,ID,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Postal Code,Founded Date,Exit Date,Website,...,All Researchers,Any Researcher,Executive Ratio,All Executives,Any Executive,Few Years Experience Ratio,Decade Experience Ratio,Mid Career Experience Ratio,Avg Gaps in Experience,avg_longest_position_duration
0,000001,2trde,"[Automotive, Software]","Munich, Bayern, Germany",2trde develops a software solution designed fo...,"57,123",0,2017-01-01,0,https://www.2trde.com,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,150.166667
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...","Munich, Bayern, Germany",36ZEROVision is an AI-powered visual inspectio...,"51,326",81671,2019-01-01,0,https://36zerovision.com/,...,1.0,1.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,7.000000,82.233333
2,000003,3Bears Foods,[Food and Beverage],"Munich, Bayern, Germany",3Bears Foods enable a balanced and delicious b...,"275,817",0,2015-01-01,0,https://3bears.de/,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]","Munich, Bayern, Germany",3dTrust helps companies integrate 3D printing ...,"134,694",80797,2015-01-01,0,http://3dtrust.de,...,0.0,1.0,0.666667,0.0,1.0,0.0,0.666667,0.333333,15.666667,99.055556
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...","Munich, Bayern, Germany",abaut builds a SaaS that enables businesses al...,"219,525",80992,2017-07-21,0,https://abaut.de,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...","Aachen, Nordrhein-Westfalen, Germany",The SAYM platform for swarm mobility defines t...,"166,910",52070.0,2019-01-01,0,https://www.saym.io/,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,13.000000,56.833333
1514,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...","Aachen, Nordrhein-Westfalen, Germany",SONAH developed a flexible embedded vision sen...,"121,752",52070.0,2016-01-01,0,http://www.sonah.tech,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1515,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]","Aachen, Nordrhein-Westfalen, Germany",Taxy.io builds the leading platform for B2B ta...,"220,816",52070.0,2019-01-01,0,https://www.taxy.io/,...,0.0,1.0,0.333333,0.0,1.0,0.0,1.000000,0.000000,2.666667,151.888889
1516,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...","Aachen, Nordrhein-Westfalen, Germany",TRINKKOST is a food supplement manufacturing c...,"907,817",0,2016-01-01,0,http://www.trinkkost.de,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


### Final Renaming of Columns


In [32]:
companies.rename(columns={
    'followers': 'LinkedIn: Followers',
    'followers_Twitter': 'X: Followers',
    'following': 'X: Following',
    'tweets': 'X: Number of Tweets',
    'account_age_days': 'X: Account Age Days',
    'tweet_activity': 'X: Tweet Activity',
    'followers_max_growth': 'X: Followers Max Growth',
    'followers_max_loss': 'X: Followers Max Loss',
    'tweets_max_growth': 'X: Tweets Max Growth',
    'tweets_max_loss': 'X: Tweets Max Loss',
'average_linkedin_followers_founders': 'LinkedIn: Average Followers Founders',
    'average_linkedin_connections_founders': 'LinkedIn: Average Connections Founders',
'min_linkedin_followers_founders': 'LinkedIn: Min Followers Founders', 'max_linkedin_followers_founders': 'LinkedIn: Max Followers Founders',
    'min_linkedin_connections_founders': 'LinkedIn: Min Connections Founders', 'max_linkedin_connections_founders': 'LinkedIn: Max Connections Founders', 'Highest Education_Bachelor': 'Highest Education Bachelor', 'Highest Education_Doctor/PhD': 'Highest Education Doctor/PhD', 'Highest Education_Master': 'Highest Education Master', 'avg_longest_position_duration': 'Avg Longest Position Duration'
''}, inplace= True)


### Final Check

In [33]:
companies = companies.drop_duplicates(subset='Organization Name', keep='first')

In [34]:
companies.to_csv('companies.csv', index=False)

In [35]:
companies

,ID,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Postal Code,Founded Date,Exit Date,Website,...,All Researchers,Any Researcher,Executive Ratio,All Executives,Any Executive,Few Years Experience Ratio,Decade Experience Ratio,Mid Career Experience Ratio,Avg Gaps in Experience,Avg Longest Position Duration
0,000001,2trde,"[Automotive, Software]","Munich, Bayern, Germany",2trde develops a software solution designed fo...,"57,123",0,2017-01-01,0,https://www.2trde.com,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,150.166667
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...","Munich, Bayern, Germany",36ZEROVision is an AI-powered visual inspectio...,"51,326",81671,2019-01-01,0,https://36zerovision.com/,...,1.0,1.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,7.000000,82.233333
2,000003,3Bears Foods,[Food and Beverage],"Munich, Bayern, Germany",3Bears Foods enable a balanced and delicious b...,"275,817",0,2015-01-01,0,https://3bears.de/,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]","Munich, Bayern, Germany",3dTrust helps companies integrate 3D printing ...,"134,694",80797,2015-01-01,0,http://3dtrust.de,...,0.0,1.0,0.666667,0.0,1.0,0.0,0.666667,0.333333,15.666667,99.055556
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...","Munich, Bayern, Germany",abaut builds a SaaS that enables businesses al...,"219,525",80992,2017-07-21,0,https://abaut.de,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...","Aachen, Nordrhein-Westfalen, Germany",The SAYM platform for swarm mobility defines t...,"166,910",52070.0,2019-01-01,0,https://www.saym.io/,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,13.000000,56.833333
1514,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...","Aachen, Nordrhein-Westfalen, Germany",SONAH developed a flexible embedded vision sen...,"121,752",52070.0,2016-01-01,0,http://www.sonah.tech,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1515,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]","Aachen, Nordrhein-Westfalen, Germany",Taxy.io builds the leading platform for B2B ta...,"220,816",52070.0,2019-01-01,0,https://www.taxy.io/,...,0.0,1.0,0.333333,0.0,1.0,0.0,1.000000,0.000000,2.666667,151.888889
1516,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...","Aachen, Nordrhein-Westfalen, Germany",TRINKKOST is a food supplement manufacturing c...,"907,817",0,2016-01-01,0,http://www.trinkkost.de,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
